### 1. Instalação das Bibliotecas e Importações

In [1]:
# Instala a biblioteca do YOLOv8 da Ultralytics.
# O '-q' no final é para uma instalação "quieta", com menos texto de log.
!pip install ultralytics -q

# Importa as bibliotecas que vamos precisar neste notebook.
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import yaml
import cv2

print("Bibliotecas instaladas e importadas com sucesso!")
print("Estamos prontos para o próximo passo.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
Bibliotecas instaladas e importadas com sucesso!
Estamos prontos para o próximo passo.


### 2. Carregar os Dados de Anotações

In [2]:
# Importa a biblioteca do Google Colab para permitir o upload de ficheiros.
from google.colab import files

# Mostra um botão para fazer o upload do ficheiro CSV que guardámos no Notebook 1.
print("Faça o upload do seu ficheiro 'road_signs_annotations.csv'")
files.upload()

# Carrega o ficheiro CSV que acabou de enviar para um DataFrame do Pandas.
# Iremos usar este DataFrame para o resto do pré-processamento.
df = pd.read_csv('road_signs_annotations.csv')

# Imprime as primeiras linhas do DataFrame para confirmar que foi carregado corretamente.
print("\nDataFrame com as anotações carregado com sucesso:")
df.head()

Faça o upload do seu ficheiro 'road_signs_annotations.csv'


Saving road_signs_annotations.csv to road_signs_annotations.csv

DataFrame com as anotações carregado com sucesso:


,filename,class,xmin,ymin,xmax,ymax
0,road793.png,speedlimit,98,192,145,241
1,road793.png,speedlimit,94,312,107,326
2,road261.png,speedlimit,201,61,236,101
3,road771.png,speedlimit,139,155,176,194
4,road535.png,speedlimit,100,254,180,334


### 3. Criar o Mapeamento de Classes

In [3]:
# Pega todos os nomes de classes únicos da coluna 'class' do DataFrame.
# .unique() retorna um array com os valores únicos, e .tolist() converte para uma lista Python.
class_names = df['class'].unique().tolist()

# Cria um dicionário de mapeamento.
# Este é um truque de Python chamado "dictionary comprehension" para criar o mapa de forma concisa.
# Para cada nome na nossa lista 'class_names', ele o associa a um número (índice).
class_map = {name: i for i, name in enumerate(class_names)}

# Imprime o mapeamento para podermos ver qual número corresponde a qual sinal.
print("Mapeamento de classes criado com sucesso:")
print(class_map)

Mapeamento de classes criado com sucesso:
{'speedlimit': 0, 'crosswalk': 1, 'trafficlight': 2, 'stop': 3}


### 4. Baixei os dados brutos

In [6]:
# Célula de Configuração para o Novo Ambiente

# Instala a biblioteca do Kaggle, caso ainda não esteja instalada
!pip install kaggle -q

# Importa a biblioteca para upload de ficheiros
from google.colab import files

# Solicita o upload do seu ficheiro kaggle.json novamente para este novo ambiente
print("Faça o upload do seu ficheiro 'kaggle.json'")
files.upload()

# Configura o diretório do Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixa o dataset novamente (este é o passo crucial)
print("\nBaixando o dataset 'Road Sign Detection' para o ambiente atual...")
!kaggle datasets download -d andrewmvd/road-sign-detection

# Descompacta o dataset, criando a pasta de imagens
print("\nDescompactando o dataset...")
!unzip -q road-sign-detection.zip -d ./road_sign_dataset

print("\n--- Ambiente Pronto!. ---")

Faça o upload do seu ficheiro 'kaggle.json'


Saving kaggle.json to kaggle (1).json

Baixando o dataset 'Road Sign Detection' para o ambiente atual...
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/road-sign-detection
License(s): CC0-1.0
road-sign-detection.zip: Skipping, found more recently modified local copy (use --force to force download)

Descompactando o dataset...
replace ./road_sign_dataset/annotations/road0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ./road_sign_dataset/annotations/road1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A

--- Ambiente Pronto!. ---


### 5. Converter Anotações para o Formato YOLO (Versão Corrigida)

In [7]:
# Define o caminho para a pasta que contém as imagens originais.
images_path = 'road_sign_dataset/images'
# Cria uma nova pasta para guardar os nossos ficheiros de anotação no formato YOLO.
# exist_ok=True evita um erro caso a pasta já exista.
labels_path = 'road_sign_dataset/labels'
os.makedirs(labels_path, exist_ok=True)

print("Iniciando a conversão das anotações para o formato YOLO...")

# O loop 'for' vai passar por cada nome de imagem único no nosso DataFrame.
for image_name in df['filename'].unique():
    # Carrega a imagem para obter a sua largura e altura.
    image_path = os.path.join(images_path, image_name)
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape

    # Filtra o DataFrame para obter apenas as anotações da imagem atual.
    annotations = df[df['filename'] == image_name]

    # Define o nome do ficheiro de anotação .txt (mesmo nome da imagem, mas com extensão .txt).
    label_filename = os.path.splitext(image_name)[0] + '.txt'
    # Abre o ficheiro de texto em modo de escrita ('w').
    with open(os.path.join(labels_path, label_filename), 'w') as f:
        # Loop 'for' para processar cada anotação (cada sinal) da imagem.
        for _, row in annotations.iterrows():
            # Obtém o ID da classe usando o mapa que criámos na célula anterior.
            class_id = class_map[row['class']]

            # Converte as coordenadas (xmin, ymin, xmax, ymax) para o formato YOLO (x_center, y_center, width, height).
            x_center = (row['xmin'] + row['xmax']) / 2
            y_center = (row['ymin'] + row['ymax']) / 2
            width = row['xmax'] - row['xmin']
            height = row['ymax'] - row['ymin']

            # Normaliza as coordenadas dividindo pela largura e altura da imagem.
            x_center_norm = x_center / img_width
            y_center_norm = y_center / img_height
            width_norm = width / img_width
            height_norm = height / img_height

            # Escreve a linha formatada no ficheiro .txt.
            f.write(f"{class_id} {x_center_norm} {y_center_norm} {width_norm} {height_norm}\n")

print(f"Conversão para o formato YOLO concluída! Foram criados {len(os.listdir(labels_path))} ficheiros de anotação na pasta 'labels'.")

Iniciando a conversão das anotações para o formato YOLO...
Conversão para o formato YOLO concluída! Foram criados 877 ficheiros de anotação na pasta 'labels'.


### 6. Dividir o Dataset em Treino e Validação

In [8]:
# Lista todos os nomes de ficheiro da nossa pasta de anotações .txt
# Usamos isto como a nossa "lista mestra" de dados válidos.
label_files = sorted([f for f in os.listdir(labels_path) if f.endswith('.txt')])

# Divide a lista de ficheiros aleatoriamente (80% para treino, 20% para validação).
# random_state=42 garante que a divisão seja sempre a mesma se executarmos o código novamente.
train_files_txt, val_files_txt = train_test_split(label_files, test_size=0.2, random_state=42)

# Cria a estrutura de pastas que o YOLO espera.
for split in ['train', 'val']:
    os.makedirs(f'dataset/{split}/images', exist_ok=True)
    os.makedirs(f'dataset/{split}/labels', exist_ok=True)

# Função auxiliar para copiar os ficheiros para as novas pastas.
def move_files(files_txt, split_name):
    print(f"Movendo {len(files_txt)} ficheiros para a pasta '{split_name}'...")
    for txt_file in files_txt:
        # Copia o ficheiro de anotação .txt
        shutil.copy(os.path.join(labels_path, txt_file), f'dataset/{split_name}/labels/')

        # Constrói o nome do ficheiro de imagem correspondente (.png)
        image_file = os.path.splitext(txt_file)[0] + '.png'
        # Copia o ficheiro de imagem
        shutil.copy(os.path.join(images_path, image_file), f'dataset/{split_name}/images/')

# Move os ficheiros de treino e validação para as suas respetivas pastas.
move_files(train_files_txt, 'train')
move_files(val_files_txt, 'val')

print("\nDivisão em treino e validação concluída com sucesso!")
print(f"Total de amostras de treino: {len(train_files_txt)}")
print(f"Total de amostras de validação: {len(val_files_txt)}")

Movendo 701 ficheiros para a pasta 'train'...
Movendo 176 ficheiros para a pasta 'val'...

Divisão em treino e validação concluída com sucesso!
Total de amostras de treino: 701
Total de amostras de validação: 176


### 7. Criar o Ficheiro de Configuração do Dataset (.yaml)

In [9]:
# Cria um dicionário Python que contém a configuração do nosso dataset.
dataset_yaml = {
    # 'train': caminho relativo para a pasta de imagens de treino.
    'train': '../dataset/train/images',

    # 'val': caminho relativo para a pasta de imagens de validação.
    'val': '../dataset/val/images',

    # 'nc': número de classes. Usamos len() para contar quantos nomes de classes únicos temos.
    'nc': len(class_names),

    # 'names': uma lista com os nomes exatos das nossas classes, na ordem correta.
    'names': class_names
}

# Define o nome do ficheiro de configuração que vamos criar.
yaml_filename = 'road_signs_dataset.yaml'

# Abre o ficheiro em modo de escrita ('w') e usa a biblioteca 'yaml' para guardar o dicionário.
with open(yaml_filename, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

# Imprime uma mensagem de sucesso para confirmar que o ficheiro foi criado.
print(f"Ficheiro de configuração '{yaml_filename}' criado com sucesso.")

Ficheiro de configuração 'road_signs_dataset.yaml' criado com sucesso.


### 8. Verificação Final

In [10]:
print("\n--- Verificação Final da Estrutura de Pastas ---")
# O comando 'ls -R' lista o conteúdo de um diretório e de todos os seus subdiretórios.
!ls -R dataset

print(f"\n--- Conteúdo do Ficheiro {yaml_filename} ---")
# O comando 'cat' exibe o conteúdo de um ficheiro de texto.
!cat {yaml_filename}

print("\n\n--- PREPARAÇÃO CONCLUÍDA! ---")
print("Tudo está pronto para o treinamento. Quando a sua GPU estiver disponível,")
print(f"o próximo notebook só precisará carregar o ficheiro '{yaml_filename}' e iniciar o treino do YOLO.")


--- Verificação Final da Estrutura de Pastas ---
dataset:
train  val

dataset/train:
images	labels

dataset/train/images:
road0.png    road236.png  road366.png  road505.png  road630.png  road761.png
road100.png  road237.png  road369.png  road506.png  road631.png  road762.png
road101.png  road238.png  road36.png   road507.png  road633.png  road764.png
road103.png  road239.png  road370.png  road508.png  road634.png  road765.png
road104.png  road23.png   road371.png  road509.png  road635.png  road767.png
road105.png  road240.png  road372.png  road50.png   road636.png  road768.png
road106.png  road241.png  road375.png  road510.png  road637.png  road769.png
road108.png  road242.png  road376.png  road511.png  road638.png  road770.png
road109.png  road243.png  road378.png  road512.png  road639.png  road771.png
road10.png   road244.png  road37.png   road513.png  road63.png	 road772.png
road110.png  road245.png  road380.png  road514.png  road640.png  road773.png
road111.png  road246.png  road3

In [11]:
# Este comando cria um ficheiro zip chamado 'dataset.zip' a partir da pasta 'dataset'
!zip -r dataset.zip dataset/

  adding: dataset/ (stored 0%)
  adding: dataset/val/ (stored 0%)
  adding: dataset/val/images/ (stored 0%)
  adding: dataset/val/images/road854.png (deflated 0%)
  adding: dataset/val/images/road206.png (deflated 0%)
  adding: dataset/val/images/road351.png (deflated 0%)
  adding: dataset/val/images/road481.png (deflated 0%)
  adding: dataset/val/images/road413.png (deflated 0%)
  adding: dataset/val/images/road783.png (deflated 0%)
  adding: dataset/val/images/road575.png (deflated 0%)
  adding: dataset/val/images/road801.png (deflated 0%)
  adding: dataset/val/images/road360.png (deflated 0%)
  adding: dataset/val/images/road107.png (deflated 0%)
  adding: dataset/val/images/road574.png (deflated 0%)
  adding: dataset/val/images/road64.png (deflated 0%)
  adding: dataset/val/images/road60.png (deflated 0%)
  adding: dataset/val/images/road198.png (deflated 0%)
  adding: dataset/val/images/road174.png (deflated 0%)
  adding: dataset/val/images/road223.png (deflated 0%)
  adding: data